## Defining Clutch: Hype Moments in the NHL

###### By:: Trevor Rowland ([dBCooper2](<https://github.com/dBCooper2>)), Abhishek Menothu ([sh4ken](<https://github.com/sh4ken>)) and Johnathen Wigfall ([jwigfall4627](<https://github.com/Jwigfall4627>))

With the time waning, the score is tied. Fans lean forward, sticks clash, and every pass is loaded
with consequence. In these last moments, players need abilities beyond skill to draw from: stamina, grit,
and the illusory ‘clutch’ factor. As fans of hockey, how is clutch defined? From that definition, how can
we rank players and teams to know which team is the most clutch, and therefore the most exciting? With
games competing in the same time slot on TV, and rising prices for tickets, how can a fan be sure that
their chosen experience will be the high-octane event that hockey is known to be? This notebook aims to
create a working definition for Clutch in the NHL, and classify teams and players who model this
definition.

Using Michael MacKelvie's video, "[The Clutch GOAT...(it's not who you think).](<https://www.youtube.com/watch?v=qjjW1l9KjXQ>)", as a springboard to start figuring out a definition of clutch in the NHL. In the video, Mackelvie uses Mike Beuoy's Clutch^2 model, which uses a Win Probability Added Model to find which shots in the NBA moved the win probability of a team more than an average amount, and those shots are called "Clutch^2". Taking this to the NHL is fairly straightforward, we just need to create a Win Probability Model for hockey games.

For this model, there is significant work being done by experts that we can draw from. For the data, there is [MoneyPuck](<https://moneypuck.com/>), which provides fantastic datasets for all things NHL, including historical shot data. This shot data will be used in conjunction with Stephen Pettigrew's [Home Team Win Probability Model](<https://cdn.prod.website-files.com/5f1af76ed86d6771ad48324b/5fee098cacad0134c9e8d403_1028_rppaper_pettigrew%20sloan%20submission.pdf>) (WP Model) to compute the win probability added by each shot. The formula for this model is:

$$
\begin{align*}

P_t(w) &= \space P_t(w \space|\space \delta_t + 1)\cdot \space \Lambda (\mathbf{\gamma_h} \cdot \mathbf{\nu_t}) \\
&+ \space P_t(w \space|\space \delta_t - 1)\cdot \space \Lambda (\mathbf{\gamma_a} \cdot \mathbf{\nu_t}) \\
&+ \space P_t(w \space | \space \delta_t) (1-\Lambda(\mathbf{\gamma_h} \cdot v_t))(1-\Lambda(\mathbf{\gamma_a} \cdot \mathbf{\nu_t}))

\end{align*}
$$

*Where...*

- $t$ is time remaining in the game.
- $P_t(w)$ is the probability of a win at time remaining, $t$.
- $\sigma_t$ is the score differential(`homeGoals` minus `awayGoals`) at time remaining, $t$.
- $\gamma_h$ is a vector of goal-scoring rates (accessed from MoneyPuck Teams Data) at different non-even strengths ([5on4, 5on3, 4on3, 3on4, 3on5, 4on5] or [5on5, other, 4on5] from the Teams dataset).
- $\gamma_a$ is the same as $\gamma_h$, only in reverse order to correspond to the away team's scoring rate ([4on5, other, 5on4]).
- $\mathbf{\nu_t}$ is a vector of seconds remaining in each of the six non-even strength situations at time $t$, using penalty times for home in 4on5, 3on5 and 3on4, and away penalty times in 5on4, 5on3 and 4on3. Unfortunately, the Moneypuck data does not provide a way to use multiple times remaining across multiple penalties, so we are assuming that we just want to use the penalty time remaining at the index of current strength for each team, and use the penalty time corresponding to when the strength of either team changes, by any amount.
- $\Lambda$ is the Poisson PMF, evaluated at $P(x=1)^2$, and will be deployed using the `scipy.stats` package.

Additionally, the term $P_t(w \space | \space \delta_t)$ is the probability that the home team will win, given a score differential $\delta_t$. This is estimated with [Alan Ryder's model](<https://www.hockeyanalytics.com/Research_files/Win_Probabilities.pdf>), and can be found on page 16 of the linked document.

## 1. Installing Packages and Importing Data

[Change later if needed!]

To run this code, you will need the polars dataframe package and scipy.stats for the poisson PMF. To install these, run:

```bash
pip install numpy polars scipy
```

In [1]:
import numpy as np
import polars as pl
from scipy import special
from math import sqrt, exp
# use for P(x=1)^2 with poisson.pmf(1, vector)
from scipy.stats import poisson

In [2]:
df_shots = pl.read_csv('/Users/dB/Desktop/fall_24/DS-4210/final-proj/final_datasets/combined_shots_2023-24.csv')
df_teams = pl.read_csv('/Users/dB/Desktop/fall_24/DS-4210/final-proj/final_datasets/combined_teams.csv')

# df_shots = df_shots.select(df_shots.columns[1:])
# df_teams = df_teams.select(df_teams.columns[1:])

## 2. Cleaning and Combining the Dataframes

We need to remove unnecessary columns from the DataFrames so they only contain features we need for the WP Model (We are no longer doing this to keep the dataset the same when it is imported into Tableau). Individual Team Scoring Rates for each season also need to be computed to create the $\mathbf{\gamma_{h}}$/$\mathbf{\gamma_{a}}$ vectors.

Many of the columns in the shots dataframe are just there for visualizing the data later in Tableau, like Shot ID's, Game ID's and the game's date. The important columns are:

- `homeTeamCode` and `awayTeamCode`: these give us the home and away team abbreviations which we will use to append the correct scoring rates to each team, and will be used later to aggregate team statistics.

- `homeTeamGoals` and `awayTeamGoals`: These are the goals each team scores within a game, and are used to find the goal differential and the win probabilities.

- `homeSkatersOnIce` and `awaySkatersOnIce`: These are used to calculate each team's strength (number of skaters on the ice) for the $\mathbf{\nu_t}$ vector.

- `homePenalty1TimeLeft` and `awayPenalty1TimeLeft`: These are used as the times for $\mathbf{\nu_t}$, and are the time left until each team's strength changes.

- `season`: season will be used for assigning the correct goal scoring percentage to a team. This model will use just 2023 and 2024 data, but is able to handle any number of seasons, as long as the data is properly formatted.

- `goal` and `isHomeTeam`: these variables say whether the shot resulted in a goal and whether that shot came from the home team, respectively. These will be used in conjunction to determine who scored a goal, and will be important for creating the win probabilities to import into tableau (this will be covered later).

In the teams data, we are extracting the following features:

- `team`: This is the team's abbreviation, and is used to match up the features with the shots dataset.

- `situation`: This describes whether the team is playing 5on5, 4on5, 5on4, or another configuration of skaters. This will be transformed into columns and used in the shots dataset for the $\mathbf{\gamma}$ vectors.

- `season`: This is just the season that the team's stats are from, and is used to match the season's stats correctly with season corresponding to a given team's shot.

- `shotOnGoalsFor` and `shotAttemptsFor`: Both of these can be used as the denominator in computing the goal-scoring rate of a team. For this analysis, shotAttemptsFor was chosen to include all shots, not just the ones on goal. This may change in future analyses.

- `goalsFor`: This is used as the numerator for the goal-scoring rate of a team.

Here is the code to clean the `shot` and `teams` data:

The following function adds goals for per game (`GFg`) and goals allowed per game (`GAg`) to the teams data.

In [3]:
# Create GFg, GAg and Scoring Rates
def add_GFg_GAg_scoring_rates_to_teams(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (pl.col("goalsFor") / pl.col("shotAttemptsFor")).alias("scoringRate"),
        (pl.col("goalsFor") / pl.col("games_played")).alias("GFg"),
        (pl.col("goalsAgainst") / pl.col("games_played")).alias("GAg")
    )

Now, with the completed teams data, we apply that data to the shots dataset.

In [4]:
# Add the Teams Data to the Shots
def merge_shots_and_teams(shots_df:pl.DataFrame, teams_df:pl.DataFrame) -> pl.DataFrame:
    # Pivot the DataFrame
    df_pivoted = teams_df.pivot(
        index=["team", "season"],  # Group by team and season
        columns="situation",      # Situations become new columns
        values=["scoringRate"]      # Values in the new columns
    )
    print(df_pivoted)

    # Rename Columns for Clarity
    df_pivoted = df_pivoted.rename(
    {col: f"scoringRate_{col}" for col in df_pivoted.columns if col not in ["team", "season"]}
    )
    
    # Now pivot with GFg and GAg here
    teams_pivoted_2 = teams_df.pivot(
        index=["team", "season"],  # Group by team and season
        columns="situation",      # Situations become new columns
        values=["GFg","GAg"]              # Values in the new columns
    )

    # Join shots with the Team data (just for home team scoring rates!)
    shots_with_home = shots_df.join(
        df_pivoted,
        left_on=["homeTeamCode", "season"],
        right_on=["team", "season"],
        how="left"#,
        #suffix="_home"
    )

    # Join away team goal efficiency with shots (adds away team scoring rates!)
    shots_with_home_and_away = shots_with_home.join(
        df_pivoted,
        left_on=["awayTeamCode", "season"],
        right_on=["team", "season"],
        how="left",
        suffix="_away"
    )

    shots_with_home_2 = shots_with_home_and_away.join(
    teams_pivoted_2,
    left_on=["homeTeamCode", "season"],
    right_on=["team", "season"],
    how="left"
    )
    shots_final_2 = shots_with_home_2.join(
        teams_pivoted_2,
        left_on=["awayTeamCode", "season"],
        right_on=["team", "season"],
        how="left",
        suffix="_away"
    )

    return shots_final_2

This is just a simple function to group all of our cleaning functions together:

In [5]:
def get_combined_df(shots_df:pl.DataFrame, teams_df:pl.DataFrame) -> pl.DataFrame:
    nu_teams_df = add_GFg_GAg_scoring_rates_to_teams(teams_df)
    df = merge_shots_and_teams(shots_df, nu_teams_df)
    return df

Running this code will return the DataFrame with all of the required data for the model. We are not finished yet, we will need to create numpy arrays for the model, but for now all the values we need are within the DataFrame.

In [6]:
df = get_combined_df(df_shots, df_teams)

shape: (522, 7)
┌──────┬────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ team ┆ season ┆ other    ┆ all      ┆ 5on5     ┆ 4on5     ┆ 5on4     │
│ ---  ┆ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str  ┆ i64    ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      │
╞══════╪════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ NJD  ┆ 2024   ┆ 0.135802 ┆ 0.056782 ┆ 0.043033 ┆ 0.083333 ┆ 0.090909 │
│ PHI  ┆ 2024   ┆ 0.131868 ┆ 0.051556 ┆ 0.040816 ┆ 0.035714 ┆ 0.064    │
│ WPG  ┆ 2024   ┆ 0.305085 ┆ 0.072529 ┆ 0.047253 ┆ 0.1      ┆ 0.145161 │
│ CAR  ┆ 2024   ┆ 0.174603 ┆ 0.056152 ┆ 0.045168 ┆ 0.066667 ┆ 0.071823 │
│ UTA  ┆ 2024   ┆ 0.173913 ┆ 0.044872 ┆ 0.038674 ┆ 0.0      ┆ 0.048387 │
│ …    ┆ …      ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
│ NYR  ┆ 2008   ┆ 0.078624 ┆ 0.04071  ┆ 0.034737 ┆ 0.064516 ┆ 0.044903 │
│ CGY  ┆ 2008   ┆ 0.102837 ┆ 0.051356 ┆ 0.044519 ┆ 0.04918  ┆ 0.063779 │
│ S.J  ┆ 2008   ┆ 0.087193 ┆ 0.0513

/var/folders/b6/zrz7f_vs0k79qphqm5sprhzr0000gn/T/ipykernel_93476/3597725823.py:4: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df_pivoted = teams_df.pivot(
/var/folders/b6/zrz7f_vs0k79qphqm5sprhzr0000gn/T/ipykernel_93476/3597725823.py:17: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  teams_pivoted_2 = teams_df.pivot(


In [7]:
df.tail()

,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,awaySkatersOnIce,awayTeamCode,awayTeamGoals,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamDefencemenOnIce,defendingTeamForwardsOnIce,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,distanceFromLastEvent,event,game_id,…,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted,composite_game_id,game_date,scoringRate_other,scoringRate_all,scoringRate_5on5,scoringRate_4on5,scoringRate_5on4,scoringRate_other_away,scoringRate_all_away,scoringRate_5on5_away,scoringRate_4on5_away,scoringRate_5on4_away,GFg_other,GFg_all,GFg_5on5,GFg_4on5,GFg_5on4,GAg_other,GAg_all,GAg_5on5,GAg_4on5,GAg_5on4,GFg_other_away,GFg_all_away,GFg_5on5_away,GFg_4on5_away,GFg_5on4_away,GAg_other_away,GAg_all_away,GAg_5on5_away,GAg_4on5_away,GAg_5on4_away
i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,str,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,i64,…,f64,f64,f64,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
150748,1219,13.0,-76.0,76.0,3.0,3.0,0.0,0,0,0,5,"""NYR""",1,41.6,42.5,28.0,41.0,28.0,28.0,2,3,44,44,28,41,28,28,41,41,28,41,28,28,1.414214,"""MISS""",20332,…,0.0322113,0.129556,0.924772,3,-3,2024020332,"""2024-11-23""",0.153846,0.041762,0.03379,0.0,0.056338,0.093333,0.059339,0.050743,0.083333,0.090909,0.5,2.75,1.85,0.0,0.4,0.35,3.1,1.95,0.8,0.0,0.411765,3.588235,2.411765,0.117647,0.647059,0.411765,2.411765,1.705882,0.294118,0.0
150749,1220,32.0,-58.0,58.0,7.0,7.0,0.0,0,0,0,5,"""NYR""",1,28.0,28.0,28.0,28.0,28.0,28.0,2,3,28,28,28,28,28,28,28,28,28,28,28,28,23.086793,"""GOAL""",20332,…,0.0259533,0.0504242,0.689601,6,-6,2024020332,"""2024-11-23""",0.153846,0.041762,0.03379,0.0,0.056338,0.093333,0.059339,0.050743,0.083333,0.090909,0.5,2.75,1.85,0.0,0.4,0.35,3.1,1.95,0.8,0.0,0.411765,3.588235,2.411765,0.117647,0.647059,0.411765,2.411765,1.705882,0.294118,0.0
150750,1221,53.0,58.0,58.0,43.0,43.0,0.0,0,0,0,5,"""NYR""",2,18.0,18.0,18.0,18.0,18.0,18.0,2,3,18,18,18,18,18,18,18,18,18,18,18,18,71.84706,"""MISS""",20332,…,0.0212277,0.026263,0.782443,41,41,2024020332,"""2024-11-23""",0.153846,0.041762,0.03379,0.0,0.056338,0.093333,0.059339,0.050743,0.083333,0.090909,0.5,2.75,1.85,0.0,0.4,0.35,3.1,1.95,0.8,0.0,0.411765,3.588235,2.411765,0.117647,0.647059,0.411765,2.411765,1.705882,0.294118,0.0
150751,1222,13.0,-78.0,78.0,7.0,7.0,-6.4,0,0,0,5,"""NYR""",2,30.2,40.0,33.0,23.666667,17.0,23.4,2,3,53,40,33,53,33,33,8,40,33,8,8,8,27.658633,"""SHOT""",20332,…,0.0368578,0.0610083,0.705245,6,-6,2024020332,"""2024-11-23""",0.153846,0.041762,0.03379,0.0,0.056338,0.093333,0.059339,0.050743,0.083333,0.090909,0.5,2.75,1.85,0.0,0.4,0.35,3.1,1.95,0.8,0.0,0.411765,3.588235,2.411765,0.117647,0.647059,0.411765,2.411765,1.705882,0.294118,0.0
150752,1223,13.0,-77.0,77.0,5.0,5.0,-6.6,0,0,0,5,"""NYR""",2,14.4,9.5,9.5,17.666667,17.666667,14.4,2,3,29,11,11,29,29,29,8,8,8,11,11,8,19.235384,"""MISS""",20332,…,0.0508426,0.0637382,0.801141,5,-5,2024020332,"""2024-11-23""",0.153846,0.041762,0.03379,0.0,0.056338,0.093333,0.059339,0.050743,0.083333,0.090909,0.5,2.75,1.85,0.0,0.4,0.35,3.1,1.95,0.8,0.0,0.411765,3.588235,2.411765,0.117647,0.647

Now we need to take this dataframe and apply the rest of the columns we need for the model. The following cells will be functions, as this needs to be re-applied to the complete dataset after testing. The first thing we need to do is remove OT shots, and create a time remaining variable that is counting down, instead of the moneypuck timer that counts up to 3600. Additionally, we will compute the goal differential here to use in the models later.

In [8]:
def remove_ot(df:pl.DataFrame) -> pl.DataFrame:
    filtered_df = df.filter(df["time"] <= 3600)
    return filtered_df

def add_time_remaining(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (3600 - df["time"]).alias("t")  # Create a new column 't'
    )

def add_goal_differential(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        (df['homeTeamGoals'] - df['awayTeamGoals']).alias('goal_delta')
    )

## Creating the Vectors $\mathbf{\gamma_h}$, $\mathbf{\gamma_a}$ and $\mathbf{\nu_t}$

The last step in creating the model is adding the `numpy` arrays that store the scoring rates at different strengths and the time remaining until strength changes. This requires us to lump the scoring rates into a single vector, and perform some logic to create the penalty time remaining vector.

In [9]:
def create_scoring_rate_vectors(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns([
        pl.concat_list([
            pl.col('scoringRate_5on4'),
            pl.col('scoringRate_other'),
            pl.col('scoringRate_4on5')
        ]).alias('y_h'),

        pl.concat_list([
            pl.col('scoringRate_4on5_away'),
            pl.col('scoringRate_other_away'),
            pl.col('scoringRate_5on4_away')
        ]).alias('y_a')
    ])

In [10]:
def create_time_remaining_vectors(df: pl.DataFrame) -> pl.DataFrame:
    """
    Creates a new column 'time_remaining_vector' in the DataFrame which contains a NumPy array of length 3
    based on the game conditions for each row.
    """
    def create_vector(rows):
        # Initialize a numpy array of length 3 with zeros
        time_remaining_vector = np.zeros((len(rows), 3), dtype=np.float64)

        for i, row in enumerate(rows):
            # Extract relevant columns from the row
            homeSkatersOnIce = row["homeSkatersOnIce"]
            awaySkatersOnIce = row["awaySkatersOnIce"]
            homePenalty1TimeLeft = row["homePenalty1TimeLeft"]
            awayPenalty1TimeLeft = row["awayPenalty1TimeLeft"]

            # Handle case 1: homeSkatersOnIce == 5 and awaySkatersOnIce == 4
            if homeSkatersOnIce == 5 and awaySkatersOnIce == 4:
                time_remaining_vector[i, 0] = float(awayPenalty1TimeLeft)

            # Handle case 2: homeSkatersOnIce == 4 and awaySkatersOnIce == 5
            elif homeSkatersOnIce == 4 and awaySkatersOnIce == 5:
                time_remaining_vector[i, 2] = float(homePenalty1TimeLeft)

            # Handle case 3: homeSkatersOnIce != awaySkatersOnIce and neither team has 5 skaters
            elif homeSkatersOnIce != awaySkatersOnIce and homeSkatersOnIce != 5 and awaySkatersOnIce != 5:
                if homePenalty1TimeLeft >= awayPenalty1TimeLeft:
                    time_remaining_vector[i, 1] = float(homePenalty1TimeLeft)
                else:
                    time_remaining_vector[i, 1] = float(awayPenalty1TimeLeft)

        return time_remaining_vector

    # Apply the function to the DataFrame
    df = df.with_columns(
        pl.struct([
            "homeSkatersOnIce",
            "awaySkatersOnIce",
            "homePenalty1TimeLeft",
            "awayPenalty1TimeLeft"
        ])
        .map_batches(create_vector)
        .alias("time_remaining_vector")
    )

    return df

# TODO: REDO, this feels like too much and its all AI generated

Now we need to apply each of these functions to the dataset:

In [11]:
def add_remaining_cols(df:pl.DataFrame) -> pl.DataFrame:
    df = remove_ot(df)
    df = add_time_remaining(df)
    df = add_goal_differential(df)
    # print("before")
    # print(df)
    df = create_scoring_rate_vectors(df)
    # print("after")
    # print(df)
    df = create_time_remaining_vectors(df)
    return df

In [12]:
df = add_remaining_cols(df)
df

,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,awaySkatersOnIce,awayTeamCode,awayTeamGoals,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamDefencemenOnIce,defendingTeamForwardsOnIce,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,distanceFromLastEvent,event,game_id,…,composite_game_id,game_date,scoringRate_other,scoringRate_all,scoringRate_5on5,scoringRate_4on5,scoringRate_5on4,scoringRate_other_away,scoringRate_all_away,scoringRate_5on5_away,scoringRate_4on5_away,scoringRate_5on4_away,GFg_other,GFg_all,GFg_5on5,GFg_4on5,GFg_5on4,GAg_other,GAg_all,GAg_5on5,GAg_4on5,GAg_5on4,GFg_other_away,GFg_all_away,GFg_5on5_away,GFg_4on5_away,GFg_5on4_away,GAg_other_away,GAg_all_away,GAg_5on5_away,GAg_4on5_away,GAg_5on4_away,t,goal_delta,y_h,y_a,time_remaining_vector
i64,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,str,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,i64,…,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,list[f64],list[f64],"array[f64, 3]"
0,0,39.698866,59.0,59.0,-26.0,26.0,-3.4,0,0,0,5,"""NSH""",0,30.0,32.0,26.0,28.666667,26.0,26.0,2,3,33,33,26,31,26,26,26,31,26,26,26,26,56.035703,"""SHOT""",20001,…,2023020001,"""2023-10-10""",0.148276,0.058241,0.044606,0.045045,0.098837,0.126712,0.051085,0.041379,0.066667,0.071895,0.52439,3.512195,2.097561,0.060976,0.829268,0.45122,3.256098,2.317073,0.426829,0.060976,0.45122,3.243902,2.04878,0.073171,0.670732,0.463415,3.02439,1.902439,0.609756,0.04878,3539,0,"[0.098837, 0.148276, 0.045045]","[0.066667, 0.126712, 0.071895]","[0.0, 0.0, 0.0]"
1,1,11.313708,81.0,81.0,8.0,8.0,-3.4,0,0,0,5,"""NSH""",0,39.0,41.0,35.0,37.666667,35.0,35.0,2,3,42,42,35,40,35,35,35,40,35,35,35,35,40.496913,"""SHOT""",20001,…,2023020001,"""2023-10-10""",0.148276,0.058241,0.044606,0.045045,0.098837,0.126712,0.051085,0.041379,0.066667,0.071895,0.52439,3.512195,2.097561,0.060976,0.829268,0.45122,3.256098,2.317073,0.426829,0.060976,0.45122,3.243902,2.04878,0.073171,0.670732,0.463415,3.02439,1.902439,0.609756,0.04878,3530,0,"[0.098837, 0.148276, 0.045045]","[0.066667, 0.126712, 0.071895]","[0.0, 0.0, 0.0]"
2,2,45.343136,55.0,55.0,30.0,30.0,-3.8,0,0,0,5,"""NSH""",0,23.8,26.0,26.0,22.333333,22.333333,23.8,2,3,26,26,26,25,25,26,19,26,26,19,19,19,120.432554,"""SHOT""",20001,…,2023020001,"""2023-10-10""",0.148276,0.058241,0.044606,0.045045,0.098837,0.126712,0.051085,0.041379,0.066667,0.071895,0.52439,3.512195,2.097561,0.060976,0.829268,0.45122,3.256098,2.317073,0.426829,0.060976,0.45122,3.243902,2.04878,0.073171,0.670732,0.463415,3.02439,1.902439,0.609756,0.04878,3493,0,"[0.098837, 0.148276, 0.045045]","[0.066667, 0.126712, 0.071895]","[0.0, 0.0, 0.0]"
3,3,43.139309,58.0,58.0,-30.0,30.0,-3.4,0,0,0,5,"""NSH""",0,40.0,45.0,45.0,36.666667,36.666667,40.0,2,3,47,46,46,47,47,47,26,44,44,26,26,26,21.213203,"""SHOT""",20001,…,2023020001,"""2023-10-10""",0.148276,0.058241,0.044606,0.045045,0.098837,0.126712,0.051085,0.041379,0.066667,0.071895,0.52439,3.512195,2.097561,0.060976,0.829268,0.45122,3.256098,2.317073,0.426829,0.060976,0.45122,3.243902,2.04878,0.073171,0.670732,0.463415,3.02439,1.902439,0.609756,0.04878,3423,0,"[0.098837

Now we have a finished dataset! Time to apply the Pettigrew Model to the Dataset. We found issues using the Pythagenpuck, Pythagenport and Ryder models, so we will instead be using the `Basic Competing Poisson` model to estimate our Win Probabilities with a given goal differential.

## 3. The Model

### 3.a. Basic Competing Poisson Model

In [13]:
# Working!!
def competing_poisson_helper(delta, GFg, GAg):
    part1 = exp(-(GFg + GAg))
    part2 = (GFg / GAg) ** (delta/2)

    # Working version of using Modified Bessel function of first order
    # (verified against Excel's BESSELI)
    x = 2 * sqrt(GFg * GAg)
    n = abs(delta)
    part3 = special.iv(n,x)

    prob = part1 * part2 * part3
    return prob

In [14]:
# Working!!
def win_prob_competing_poisson(delta, GFg, GAg):
    num = competing_poisson_helper(delta, GFg, GAg)
    denom = competing_poisson_helper(delta, GFg, GAg) + competing_poisson_helper(-delta, GFg, GAg)
    prob = num/denom
    return prob

### 3.b. The Pettigrew Win Probability Model

Finally, here is the final implementation of the Pettigrew Win Probability Model in Python, using the Basic Competing Poisson Estimation of Ryder Win Probabilities. This function will run the Pettigrew Model using the entire dataset, add win probability and clutch^2 columns classifying each shot, and return the resultant DataFrame.

In [15]:
def add_wp_and_clutch2(df: pl.DataFrame, wp_method=None, use_pettigrew: bool=True) -> pl.DataFrame:
    """
    Compute Win Probabilities and Clutch² classification for each row in the DataFrame.

    Parameters:
    -----------
    df : pl.DataFrame
        Input DataFrame with game and shot information

    Returns:
    --------
    pl.DataFrame
        DataFrame with added Win Probability and Clutch² columns
    """
    # Ensure the DataFrame is sorted by game and time
    df_sorted = df.sort(["composite_game_id", "time"])

    # Compute Win Probabilities and Clutch2 classification
    def process_game(game_df):
      # Initialize lists to store Win Probabilities
      wp_before = []
      wp_after = []
      goal_flags = []

      for i in range(len(game_df)):
        row = game_df.row(i, named=True)

        # redesigning this logic is hurting my head
        # going to use this space to extract whichever row values are potentially needed
        # and then have the win_prob module wrapper take care of which to actually send through

        chosen_wp_method = wp_method

        isHomeTeam = row['isHomeTeam'] == 1

        # computing goal_diff with respect to the home team
        goal_diff = row['homeTeamGoals'] - row['awayTeamGoals']

        # determining status of home team before the shot
        if goal_diff > 0:
          homeStatus = 'leading'
        elif goal_diff < 0:
          homeStatus = 'behind'
        else:
          homeStatus = 'tied'

        wp_b = calc_win_prob(
            chosen_wp_method,
            isHomeTeam = isHomeTeam,
            homeTeamStatus = homeStatus,
            homeTeamGoals = row['homeTeamGoals'],
            awayTeamGoals = row['awayTeamGoals'],
            homeGFg_all = row['GFg_all'],
            awayGFg_all = row['GFg_all_away'],
            homeGAg_all = row['GAg_all'],
            awayGAg_all = row['GAg_all_away'],

            #going of Pettigrew's statement that home team has 54% chance of winning when score diff = 0
            tied_home_wp = 0.54,

            # not needed for Ryder methods but building in for future improvement/retry
            homeScoringRate = row['y_h'],
            awayScoringRate = row['y_a'],
            timeRemaining = row['time_remaining_vector']
        )

        wp_before.append(wp_b)
        goal_flags.append(row.get('goal', 0))

        # Logic to process if shot was a goal before computing wp_a
        if row.get('goal', 0) == 1 and isHomeTeam:
            homeTeamGoals_a = row['homeTeamGoals'] + 1
            awayTeamGoals_a = row['awayTeamGoals']
        elif row.get('goal', 0) == 1 and not isHomeTeam:
            homeTeamGoals_a = row['homeTeamGoals']
            awayTeamGoals_a = row['awayTeamGoals'] + 1
        else:
            homeTeamGoals_a = row['homeTeamGoals']
            awayTeamGoals_a = row['awayTeamGoals']

        # recomputing goal_diff with respect to the home team
        goal_diff_a = homeTeamGoals_a - awayTeamGoals_a

        # redetermining status of home team after the shot
        if goal_diff_a > 0:
          homeStatus_a = 'leading'
        elif goal_diff < 0:
          homeStatus_a = 'behind'
        else:
          homeStatus_a = 'tied'

        wp_a = calc_win_prob(
            chosen_wp_method,
            isHomeTeam = isHomeTeam,
            homeTeamStatus = homeStatus_a,
            homeTeamGoals = homeTeamGoals_a,
            awayTeamGoals = awayTeamGoals_a,
            homeGFg_all = row['GFg_all'],
            awayGFg_all = row['GFg_all_away'],
            homeGAg_all = row['GAg_all'],
            awayGAg_all = row['GAg_all_away'],

            #going of Pettigrew's statement that home team has 54% chance of winning when score diff = 0
            tied_home_wp = 0.54,

            # not needed for Ryder methods but building in for future improvement/retry
            homeScoringRate = row['y_h'],
            awayScoringRate = row['y_a'],
            timeRemaining = row['time_remaining_vector']
        )

        wp_after.append(wp_a)

      # Compute Win Probability changes
      wp_changes = np.array(wp_after) - np.array(wp_before)
      avg_wp_change = np.mean(np.abs(wp_changes))
      # TODO: interrogate the logic of abs()
      # would it make more sense to avg out the positive wp_changes since that's what we are interested in?

      # Create a new DataFrame with computed values
      clutch_df = game_df.with_columns([
            pl.Series(name='wp_before', values=wp_before),
            pl.Series(name='wp_after', values=wp_after),
            pl.Series(name='wp_change', values=wp_changes),
            pl.Series(name='is_clutch2', values=wp_changes > avg_wp_change),
            pl.Series(name='goal_flag', values=goal_flags)
      ])
      return clutch_df

    def calc_win_prob(wp_method, **kwargs):
      """
      Uses row values in kwargs to determine how to pass args into the chosen wp_method.
      Designed to be flexible where you can construct more complicated wp_methods in the future
      and need little to no modification to swap in

      Returns a single float win probability
      """
      if use_pettigrew:
        #Win Probs for if Home team
        if kwargs['homeTeamStatus'] == 'tied':
          wp = kwargs['tied_home_wp']

        else:
          wp = pettigrew_wp(
              delta = kwargs['homeTeamGoals'] - kwargs['awayTeamGoals'],
              GFg = kwargs['homeGFg_all'],
              GAg = kwargs['homeGAg_all'],
              scoring_rate_home=kwargs['homeScoringRate'],
              scoring_rate_away=kwargs['awayScoringRate'],
              times_remaining=kwargs['timeRemaining']
          )

        # Handling if Away team
        if not kwargs['isHomeTeam']:
          wp = 1-wp

        return wp

      else:
        wp_kwargs = kwargs.copy()
        # Win Probs for if tied
        if kwargs['homeTeamStatus'] == 'tied' and kwargs['isHomeTeam']:
          wp = kwargs['tied_home_wp']
        elif kwargs['homeTeamStatus'] == 'tied' and not kwargs['isHomeTeam']:
          wp = 1 - kwargs['tied_home_wp']

        # Win Probs for if Home is leading
        elif kwargs['homeTeamStatus'] == 'leading' and kwargs['isHomeTeam']:
          wp = wp_method(
              delta = kwargs['homeTeamGoals'] - kwargs['awayTeamGoals'], #needs to > 0
              GFg = kwargs['homeGFg_all'],
              GAg = kwargs['homeGAg_all']
          )
        elif kwargs['homeTeamStatus'] == 'leading' and not kwargs['isHomeTeam']:
          wp = 1 - wp_method(
              delta = kwargs['homeTeamGoals'] - kwargs['awayTeamGoals'], #needs to > 0
              GFg = kwargs['homeGFg_all'],
              GAg = kwargs['homeGAg_all']
          )

        # Win Probs for if Home is behind
        elif kwargs['homeTeamStatus'] == 'behind' and kwargs['isHomeTeam']:
          wp = 1 - wp_method(
              delta = kwargs['awayTeamGoals'] - kwargs['homeTeamGoals'], #needs to > 0
              GFg = kwargs['awayGFg_all'],
              GAg = kwargs['awayGAg_all']
          )
        elif kwargs['homeTeamStatus'] == 'behind' and not kwargs['isHomeTeam']:
          wp = wp_method(
              delta = kwargs['awayTeamGoals'] - kwargs['homeTeamGoals'], #needs to > 0
              GFg = kwargs['awayGFg_all'],
              GAg = kwargs['awayGAg_all']
          )
        return wp

    # Process each game group
    result_dfs = []
    for game_id, game_df in df_sorted.group_by("composite_game_id"):
        processed_game_df = process_game(game_df)
        result_dfs.append(processed_game_df)

    # Combine the results and sort
    return (pl.concat(result_dfs)
            .sort(["composite_game_id", "time"])
    )

In [16]:
def poisson_wp(gamma, nu):
    # gamma and nu are vectors
    dotprod = np.dot(gamma, nu, out=None)
    pmf_val = poisson.pmf(1, dotprod)
    return pmf_val

Finally, we can call the function to add the win probabilities and clutch^2 values to the DataFrame, and write it to a CSV to import into Tableau.

In [17]:
result = add_wp_and_clutch2(
    df = df,
    wp_method = None,
    use_pettigrew = True
)

NameError: name 'pettigrew_wp' is not defined

In [ ]:
result = result.drop(["y_a", "y_h", "time_remaining_vector"]) # Drop these vectors, they don't play nice outside of a dataframe
result.write_csv('/path/to/save/output/final_shots.csv')

## 4. Next Steps

The model is working as expected, and from the dashboard we are seeing accurate results. However, this model was never tested for accuracy, so the next steps are to manually identify shots that would be considered by audiences as "Clutch" and then comparing our Clutch^2 predictions to assess the accuracy of the model. 

Additionally, we could update the model to factor in defensive plays like Corsi and blocked shots to better examine clutch defensive plays. We may need to make this into its own model however, so stay tuned for that!

Lastly, the Pettigrew model is a very simple implementation of Win Probability. This model could be taken further with Pettigrew's Added Goal Value metric outlined in the paper, and could be made stricter to better classify Clutch^2 shots, or made more comprehensive through an overtime handler.

These better models and a better testing methodology could lead the team to using Machine Learning to either improve the model, or scrap the Pettigrew model entirely for a widely-used model like `XGBoost` and a Random Forest for identifying important features of the model.

Overall this was an incredibly interesting and motivating project to work on, and we are excited to see where this will lead.

## 4. Comparing Clutch^2 Shots to Game-Winning Goals

To assess the accuracy of the model, we are comparing our classified shots to the initial clutch metric. If a clutch^2 shot is a game-winning goal, then that is considered an accurate classification.